In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- CẤU HÌNH ---
FILE_PATH = os.path.join('..', 'data', 'raw', 'dataset_raw.csv')

try:
    # 1. Đọc dữ liệu (thêm encoding='utf-8-sig' để đọc tiếng Việt không lỗi)
    if FILE_PATH.endswith('.xlsx'):
        df = pd.read_excel(FILE_PATH)
    else:
        df = pd.read_csv(FILE_PATH, encoding='utf-8-sig')

    print("✅ Đã đọc file thành công!")

    # 2. Xử lý sơ bộ (Xóa dòng trống nếu có)
    # Loại bỏ các dòng mà cột label hoặc comment_text bị trống
    df = df.dropna(subset=['label', 'comment_text'])

    # 3. MAPPING NHÃN (Tự động chuyển chữ sang số để vẽ)
    # Chuẩn hóa text trong cột label (xóa khoảng trắng thừa đầu đuôi)
    df['label'] = df['label'].astype(str).str.strip()

    # Dictionary ánh xạ (Dựa trên screenshot và tài liệu của bạn)
    label_mapping = {
        'Không xúc phạm': 0,
        'Mỉa mai': 1,
        'Xúc phạm': 2
    }

    # Tạo cột label_id mới (dạng số)
    df['label_id'] = df['label'].map(label_mapping)

    # Kiểm tra xem có nhãn nào bị sót không (VD: gõ sai chính tả)
    missing_labels = df[df['label_id'].isna()]['label'].unique()
    if len(missing_labels) > 0:
        print(f"⚠️ CẢNH BÁO: Các nhãn sau không map được (kiểm tra chính tả): {missing_labels}")
        # Tạm thời điền -1 cho các nhãn lỗi để code không crash
        df['label_id'] = df['label_id'].fillna(-1)

    # --- VẼ BIỂU ĐỒ 1: PHÂN BỐ NHÃN (LABEL DISTRIBUTION) ---
    plt.figure(figsize=(8, 6))
    ax = sns.countplot(x='label_id', data=df, palette='viridis')
    
    plt.title('Phân bố nhãn (0: Non-shaming, 1: Sarcastic, 2: Offensive)')
    plt.xlabel('Nhãn (Label)')
    plt.ylabel('Số lượng (Count)')
    plt.xticks([0, 1, 2], ['0: Không xúc phạm', '1: Mỉa mai', '2: Xúc phạm']) # Gắn nhãn trục X cho dễ đọc

    # Hiển thị số lượng trên cột
    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height), 
                        ha='center', va='bottom')

    plt.savefig('label_distribution.png') # Lưu ảnh
    print("✅ Đã lưu ảnh: label_distribution.png")
    plt.show()

    # --- VẼ BIỂU ĐỒ 2: ĐỘ DÀI CÂU (SENTENCE LENGTH) ---
    # Tính số từ
    df['word_count'] = df['comment_text'].apply(lambda x: len(str(x).split()))

    plt.figure(figsize=(10, 6))
    sns.histplot(df['word_count'], bins=30, kde=True, color='royalblue')
    plt.title('Phân bố độ dài câu (Theo số từ)')
    plt.xlabel('Số từ (Word Count)')
    plt.ylabel('Tần suất (Frequency)')
    
    plt.savefig('sentence_length.png') # Lưu ảnh
    print("✅ Đã lưu ảnh: sentence_length.png")
    plt.show()

    # --- IN THỐNG KÊ ĐỂ VIẾT BÁO CÁO ---
    print("\n--- SỐ LIỆU BÁO CÁO ---")
    print("Tổng số dòng dữ liệu:", len(df))
    print("\nTỷ lệ phân bố nhãn (%):")
    print(df['label_id'].value_counts(normalize=True).sort_index() * 100)
    print("\nThống kê độ dài câu:")
    print(f"- Trung bình: {df['word_count'].mean():.2f} từ")
    print(f"- Ngắn nhất: {df['word_count'].min()} từ")
    print(f"- Dài nhất: {df['word_count'].max()} từ")

except FileNotFoundError:
    print("❌ Lỗi: Không tìm thấy file. Hãy kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"❌ Lỗi không xác định: {e}")

❌ Lỗi: Không tìm thấy file. Hãy kiểm tra lại tên file và đường dẫn.
